<center><h1> IFT-6758  Data Science  </h1></center>
<center><h2> Fall - 2021 </h2></center> 
<center><h3> Team : Kevin Traoré, Etienne Delisle, Paul Villedieu, Mohammed Laredj </h3></center> 
<center><h3> Interactive Debugging Tool </h3></center> 



In [1]:
jsonDataFolder = '../jsondata'
datasetsFolder = '../datasets'
print('Please enter a desired season year, e.g. : 2018 (for 2018-2019 season)')
desiredSeason = int(input())

Please enter a desired season year, e.g. : 2018 (for 2018-2019 season)


 2019


In [2]:
from ift6758.data import goalsCoordinates, GameBuilder, get_shooter_or_scorer_and_goalie,\
        none_to_na, create_folder_if_not_exists, get_opponent_goal_coordinates
import pandas as pd
from ift6758.data.question_2 import nhl_data

In [3]:
create_folder_if_not_exists(jsonDataFolder)
create_folder_if_not_exists(datasetsFolder)

gamesList = nhl_data([desiredSeason], ['02', '03'], jsonDataFolder, light=True)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1376/1376 [00:44<00:00, 31.10it/s]


In [4]:
def load_game(gameNum):
    import json
    
    with open(f'{jsonDataFolder}/{gameNum}.json', 'r') as file:
        game_data = file.read()

    return GameBuilder().build(json.loads(game_data))


In [5]:
def get_shots_goals_events(game):
    events = game.events
    eventFilterLambda = lambda event: event.eventTypeId in ['GOAL', 'SHOT']
    return list(filter(eventFilterLambda, events))


In [6]:
def events_list_to_dataFrame(gameId, game):
    eventsList = game.events
    homeTeam = game.homeTeam
    awayTeam = game.awayTeam
    periodsDict = game.periods
    data = []
    for event in eventsList:
        shooterOrScorer, goalie = get_shooter_or_scorer_and_goalie(event.players)
        shooterTeam = event.shooterTeam
        homeTeamTriCode = homeTeam.triCode
        eventCoordinates = event.coordinates
        opponentGoal = get_opponent_goal_coordinates(shooterTeam, homeTeamTriCode,\
                                                     eventCoordinates, event.periodNumber, periodsDict)
        
        data.append({'Game' : gameId,
                     'Type' : event.eventTypeId,
                     'Secondary Type' : event.secondaryType,
                     'Period' : event.periodNumber,
                     'Period Time' : event.periodTime,
                     'Remaining Time' : event.periodRemainingTime,
                     'Home Team' : homeTeamTriCode,
                     'Away Team' : awayTeam.triCode,
                     'Home Team Shorthanded' : awayTeam.powerPlay,
                     'Away Team Shorthanded' : homeTeam.powerPlay,
                     'Shooter/Scorer Team' : shooterTeam,
                     'Shooter/Scorer' : shooterOrScorer.fullName,
                     'Goalie' : none_to_na(goalie, 'fullName'),
                     'Shooter/Scorer X Coordinate' : none_to_na(eventCoordinates, 'x'),
                     'Shooter/Scorer Y Coordinate' : none_to_na(eventCoordinates, 'y'),
                     'Opponent Goal X Coordinate' : none_to_na(opponentGoal, 'x'),
                     'Opponent Goal Y Coordinate' : none_to_na(opponentGoal, 'y'),
                     'Empty Net' : none_to_na(event, 'emptyNet')
                  })

    return pd.DataFrame(data)

In [7]:
def save_teams_tricode_and_name(tricodeAndNameDict):
    tricodeAndNameDataFrame = pd.DataFrame({'Team Tricode' : tricodeAndNameDict.keys(),
                                            'Team Name' : tricodeAndNameDict.values()})
    outputFileName = f'{datasetsFolder}/teams_tricode_and_name.csv'
    tricodeAndNameDataFrame.to_csv(outputFileName, index=None)
    print('\n', f'Dataset {outputFileName} successfully created')

In [8]:
ignoredGamesId = []
eventsDataFrame = pd.DataFrame({})
teamsNameByTricode = {}
for gameId in gamesList:
    try:
        currentGame = load_game(gameId)
        currentGame.events = get_shots_goals_events(currentGame)
        eventsDataFrame = eventsDataFrame.append(events_list_to_dataFrame(gameId, currentGame), ignore_index=True)
        
        teamsNameByTricode[currentGame.homeTeam.triCode] = currentGame.homeTeam.name
        teamsNameByTricode[currentGame.awayTeam.triCode] = currentGame.awayTeam.name
    except FileNotFoundError:
        ignoredGamesId.append(gameId)

datasetName = f'{datasetsFolder}/{desiredSeason}-{desiredSeason + 1}_season_shots_and_goals_events.csv'
eventsDataFrame.to_csv(datasetName, index=None)
save_teams_tricode_and_name(teamsNameByTricode)
print('\n', f'Dataset {datasetName} successfully created', '\n\n#########################################################\n')
print(eventsDataFrame.info(), '\n\n#########################################################\n')
print(eventsDataFrame.head(), '\n\n#########################################################\n')
eventsDataFrame.head(10).to_html(f'{datasetName[:-4]}.html')
print(eventsDataFrame.tail())


 Dataset ../datasets/teams_tricode_and_name.csv successfully created

 Dataset ../datasets/2019-2020_season_shots_and_goals_events.csv successfully created 

#########################################################

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73867 entries, 0 to 73866
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Game                         73867 non-null  object 
 1   Type                         73867 non-null  object 
 2   Secondary Type               73856 non-null  object 
 3   Period                       73867 non-null  int64  
 4   Period Time                  73867 non-null  object 
 5   Remaining Time               73867 non-null  object 
 6   Home Team                    73867 non-null  object 
 7   Away Team                    73867 non-null  object 
 8   Home Team Shorthanded        73867 non-null  bool   
 9   Away Team Shorthanded        7